In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
train = pd.read_csv('../data/train.csv')
test  = pd.read_csv('../data/test_2.csv')

In [16]:
train = train.fillna(train.median())
test = test.fillna(test.median())

In [17]:
train_ft   = train.ix[:,  1: 26]
train_X    = train.ix[:, 26: 28]
train_ts_X = train.ix[:, 28:147]
train_ts_Y = train.ix[:,147:207]
train_Y1   = train.ix[:,    207]
train_Y2   = train.ix[:,    208]
train_WI   = train.ix[:,    209]
train_WD   = train.ix[:,    210]

test_ft    = test.ix[:,   1: 26]
test_X     = test.ix[:,  26: 28]
test_ts_X  = test.ix[:,  28:147]

In [18]:
def plot_ts(ts):
    plt.plot(np.array(ts))
    plt.ylabel('return')
    plt.show()

def plot_hist(a):
    plt.hist(a, bins=50, normed=True)
    plt.show()

In [19]:
# plot_hist(train_WD)

In [20]:
sub = pd.read_csv('../data/sample_submission_2.csv')

In [21]:
from sklearn.neighbors import KNeighborsRegressor

# create an KNN model
def knn(X, y):
    m = KNeighborsRegressor(n_neighbors=50, weights='distance', n_jobs=4)
    m.fit(X, y)
    return m

In [22]:
# daily weights
m_WD = knn(train_ft, train_WD)
test_WD = 1.0 / m_WD.predict(test_ft)**4 # reverse weight
test_WD = test_WD / test_WD.mean() # normalize

In [10]:
# m_Y1 = knn(train_ft, train_Y1)
# m_Y2 = knn(train_ft, train_Y2)
# test_Y1 = m1.predict(test_ft)
# test_Y2 = m2.predict(test_ft)

In [11]:
# t_y1 = test_Y1 * test_WD / test_WD.mean()
# t_y2 = test_Y2 * test_WD / test_WD.mean()

In [23]:
# Daily prediction
# weighted median method
t_y1 = train_Y1.median() * test_WD
t_y2 = train_Y2.median() * test_WD
sub.loc[60::62,'Predicted'] = t_y1
sub.loc[61::62,'Predicted'] = t_y2

In [13]:
# Intraday prediction
WR = 1.25
test_WI = test_WD / WR
ts_md = train_ts_Y.median()
for i in range(0, ts_md.shape[0]):
    sub.loc[i::62,'Predicted'] = ts_md[i] * test_WI

In [24]:
sub.to_csv('../data/my_sub.csv', index=False)